This notebook takes the files that were not used to create the word list in the find_words notebook and vectorizes them so they can be fed through models. Still working on a way to effectively safe this numerical data, but I may only be able to do that with binary data

In [5]:
import os
import json
import numpy as np
import pandas as pd
import pyarrow as pa
from pandas.io.json import json_normalize
import cryptography
import re
from cryptography.fernet import Fernet
import bs4
from nltk.tokenize import word_tokenize
import pickle
from collections import Counter

In [6]:
def save_obj(obj, name ):
    with open(name + '.pkl', 'wb') as f:
        pickle.dump(obj, f, pickle.HIGHEST_PROTOCOL)

def load_obj(name ):
    with open(name + '.pkl', 'rb') as f:
        return pickle.load(f)
    
def tokenize(text):
    text = re.sub('&quot;','\'',text)
    text = re.sub('&amp;','&',text)
    text = re.sub('"', '', text)
    text = re.sub(r'\'(?!\')(.*)\'',r'\0', text)
    
    toks = re.split(r'[\s,:;{}\(\)<>\/�]+',text.lower())
    return sorted(toks)

In [7]:
vocab = [a for a,b in load_obj('sorted_kvpairs')]
vocab = sorted(vocab[:10000])#takes the top 10000 by freq and puts them in alphabetical/ascii order

In [29]:
# Create corpus
data = []
labels = []
indicators=[]
# Not included as all their files seem to be empty or they have no enrichment: 
    #4/5, 4/6, 4/13, 4/14, 4/15, 4/31, 5/13, 5/14, 5/15, 5/16, 5/17, 5/18, 7/4, 7/5, 7/6
# Empty List serves as a place holder between the data used in find_words and the test data I'm using here
date_batches = [['newbenigndata'],['2019-04-22','2019-04-23','2019-04-24','2019-04-25','2019-04-26','2019-04-27','2019-04-28','2019-04-29'],
                ['2019-04-11','2019-04-12','2019-04-16','2019-04-17','2019-04-18','2019-04-19','2019-04-20','2019-04-21'],
                ['2019-04-30','2019-05-01','2019-05-02','2019-05-03','2019-05-04'],
                ['2019-05-05','2019-05-06','2019-05-07','2019-05-08','2019-05-09','2019-05-10','2019-05-11','2019-05-12'],
                ['2019-05-19','2019-05-20'],
                ['2019-05-21','2019-05-22','2019-05-23','2019-05-24','2019-05-25','2019-05-26','2019-05-27','2019-05-28'],
                ['2019-05-29','2019-05-30','2019-05-31','2019-06-01','2019-06-02','2019-06-03','2019-06-04','2019-06-05'],
                ['2019-06-06','2019-06-07','2019-06-08','2019-06-09','2019-06-10','2019-06-11','2019-06-12','2019-06-13'],[],
                ['2019-06-14', '2019-06-15','2019-06-16', '2019-06-17','2019-06-18','2019-06-19', '2019-06-20', '2019-06-21'],
                ['2019-06-22', '2019-06-23', '2019-06-25', '2019-06-26', '2019-06-27', '2019-06-28','2019-06-29', '2019-06-30'],
                ['2019-07-01','2019-07-02', '2019-07-03', '2019-07-07'],
                ['newNONTORbenigndata']]
dates = date_batches[-1]+date_batches[-2]+date_batches[-3]+date_batches[-4]#+date_batches[3]+date_batches[4]+date_batches[5]+date_batches[6]+date_batches[7]+date_batches[8] 
#'2019-04-12','2019-04-16','2019-04-17','2019-04-18','2019-04-19','2019-04-20','2019-04-21]
for date in dates:
    files = os.listdir('/data/data/'+date+'/enrichment/fetch_page')
    top = len(files)
    if top > 70:
        top = 70
    for j in range(top):
        with open('/data/data/'+date+'/enrichment/fetch_page/'+files[j]) as d:
            r = json.load(d)
            try:
                df = json_normalize(r['data'])
            except:
                break
        try:
            df = df[df['status_code'] == 200].reset_index(drop=True)
        except:
            df = df[df['success']]
        #Need to determine label
        html_str = df['page_content'].copy()
        
        for i in range(len(html_str)):
            if df.loc[i,'indicator'] in indicators:
                continue
            else:
                indicators.append(df.loc[i, 'indicator'])
            if len(files[j]) > 10:
                labels += [1]
            else:
                labels += [0]
            key = bytes( df.loc[i,'encryption_key'],encoding = 'UTF-8')
            f = Fernet(key) 
            #decrypt string
            text = f.decrypt(bytes(html_str[i],encoding='UTF-8'))
            #clean out html
            text = text.decode('utf8')
            toks = tokenize(text)
            #need to create vector 
            count = Counter()
            toks = set(toks)
            for tok in toks:
                count[tok] += 1
            vec = []
            for v in vocab:
                vec = np.append(vec, [count[v]])
            data.append(vec)
                
            
            
         #end of for
    #end of for
    print('------------'+date+'---------------')
#end of for
data = np.row_stack(data)
data[:2]

------------newNONTORbenigndata---------------
------------2019-07-01---------------
------------2019-07-02---------------
------------2019-07-03---------------
------------2019-07-07---------------
------------2019-06-22---------------
------------2019-06-23---------------
------------2019-06-25---------------
------------2019-06-26---------------
------------2019-06-27---------------
------------2019-06-28---------------
------------2019-06-29---------------
------------2019-06-30---------------
------------2019-06-14---------------
------------2019-06-15---------------
------------2019-06-16---------------
------------2019-06-17---------------
------------2019-06-18---------------
------------2019-06-19---------------
------------2019-06-20---------------
------------2019-06-21---------------


array([[1., 0., 0., ..., 0., 0., 0.],
       [1., 0., 0., ..., 0., 0., 0.]])

In [30]:
print(data.shape)
np.column_stack([data,labels]).shape

(4440, 10000)


(4440, 10001)

In [10]:
from sklearn.ensemble import RandomForestClassifier
from sklearn.ensemble import RandomForestRegressor
from sklearn.metrics import classification_report
from sklearn.model_selection import train_test_split

In [31]:
X = pd.DataFrame( data,columns=vocab)#np.column_stack([data,labels])
X['unsafe'] = labels
X1 = X[X['unsafe']==1].sample(frac=1)
print(X1.shape)
X0 = X[X['unsafe']==0]
print(X0.shape)
X1 = X1.reset_index(drop=True).loc[:(X0.shape[0]),:]
print(X1.shape)
temp_df = pd.concat([X0,X1]).sample(frac=1)
X = temp_df.drop(columns=['unsafe'])
Y = temp_df['unsafe']

(2416, 10001)
(2024, 10001)
(2025, 10001)


In [13]:
X.head()

, ,  ,   ,    ,     ,      ,       ,        ,            ,              ,...,,,,,洛,﻿,𱤹ǰ,񆎰,󿬅,􎉴[`kx
533,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
567,1.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
590,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
251,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
695,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0


In [32]:
#X = data
#Y = labels
X_train, X_test, Y_train, Y_test = train_test_split(X, Y, train_size= 0.5, test_size=0.5, random_state=13)
forest = RandomForestClassifier(n_estimators=80,random_state=13)
forest.fit(X_train,Y_train)
print()
print(forest.score(X_test,Y_test))
print(*(sorted(list(zip(vocab,forest.feature_importances_)), key=lambda x: x[1],reverse=True))[:100], sep = '\n')
print(classification_report(Y_test,forest.predict(X_test)))

forest = RandomForestRegressor(n_estimators=80,random_state=13)
forest.fit(X_train,Y_train)
print(forest.score(X_test,Y_test))
print(*(sorted(list(zip(vocab,forest.feature_importances_)), key=lambda x: x[1],reverse=True))[:100], sep = '\n')



0.8720987654320987
('.gettime', 0.007211518004171516)
('name=description', 0.006964242608226962)
('href=https', 0.006883216882838714)
('script', 0.005643908890702067)
('new', 0.005445354979575827)
('s', 0.005336315271883909)
('||', 0.005226819611805348)
('hidden', 0.005191186795500147)
('w', 0.004951281648809325)
('\x00', 0.004854739156144193)
('date', 0.004749199584284138)
('style=display', 0.0046991893459875245)
('13', 0.004670589557140938)
('[]', 0.004586772419947345)
('10', 0.004415773903514908)
('span', 0.004214632501993295)
('[0]', 0.004162046450526997)
('property=fb', 0.003974566316996299)
('target=_blank', 0.003972647003254523)
('2019', 0.0037979135797544597)
('!doctype', 0.003738887617814578)
('|', 0.0035601806450565677)
('url', 0.0035507324140240672)
('complete===b.readystate&&c.readycallback', 0.0033807259121708434)
('image', 0.003337402382259342)
('function', 0.0031661923576471315)
('false', 0.0030964688346318686)
('s.w.org\\', 0.0029543226588493493)
('1', 0.00289838714924

In [33]:
from sklearn.linear_model import LogisticRegression
clf = LogisticRegression(random_state=13)
clf.fit(X_train,Y_train)
print(clf.score(X_test,Y_test))
print(classification_report(Y_test,clf.predict(X_test)))

0.8864197530864197
             precision    recall  f1-score   support

          0       0.92      0.85      0.89      1038
          1       0.86      0.92      0.89       987

avg / total       0.89      0.89      0.89      2025



In [34]:
from sklearn.neural_network import MLPClassifier
clf = MLPClassifier(verbose=True, batch_size= 11, random_state = 13)
clf.fit(X_train,Y_train)
print(clf.score(X_test,Y_test))
print(classification_report(Y_test,clf.predict(X_test)))

Iteration 1, loss = 0.40684114
Iteration 2, loss = 0.16695831
Iteration 3, loss = 0.09421176
Iteration 4, loss = 0.06579691
Iteration 5, loss = 0.05165654
Iteration 6, loss = 0.05030142
Iteration 7, loss = 0.04644707
Iteration 8, loss = 0.05243231
Iteration 9, loss = 0.06037857
Iteration 10, loss = 0.06852215
Training loss did not improve more than tol=0.000100 for two consecutive epochs. Stopping.
0.8725925925925926
             precision    recall  f1-score   support

          0       0.95      0.80      0.87      1038
          1       0.82      0.95      0.88       987

avg / total       0.88      0.87      0.87      2025



In [28]:
clf = MLPClassifier(verbose=True, batch_size= 12, random_state = 13)
clf.fit(X_train,Y_train)
print(clf.score(X_test,Y_test))
print(classification_report(Y_test,clf.predict(X_test)))

Iteration 1, loss = 0.42246430
Iteration 2, loss = 0.19735366
Iteration 3, loss = 0.12187240
Iteration 4, loss = 0.07751181
Iteration 5, loss = 0.05450115
Iteration 6, loss = 0.04955718
Iteration 7, loss = 0.04400787
Iteration 8, loss = 0.04256621
Iteration 9, loss = 0.03743643
Iteration 10, loss = 0.04482492
Iteration 11, loss = 0.04163082
Iteration 12, loss = 0.03904210
Training loss did not improve more than tol=0.000100 for two consecutive epochs. Stopping.
0.8784909358157765
             precision    recall  f1-score   support

          0       0.94      0.81      0.87      1023
          1       0.83      0.94      0.89      1018

avg / total       0.89      0.88      0.88      2041

